In [1]:
import torch
from selfcheckgpt.modeling_mqag import MQAG

In [2]:
torch.manual_seed(28)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
mqag_model = MQAG(
    g1_model_type='race', # race (more abstractive), squad (more extractive)
    device=device
)

MQAG (race) initialized to cuda


# MQAG Score: `score`
- Using MQAG for assessing the consistency between document and summary as described in: https://arxiv.org/abs/2301.12307

In [5]:
document = r"""
World number one Novak Djokovic says he is hoping for a "positive decision" to allow him
to play at Indian Wells and the Miami Open next month. The United States has extended
its requirement for international visitors to be vaccinated against Covid-19. Proof of vaccination
will be required to enter the country until at least 10 April, but the Serbian has previously
said he is unvaccinated. The 35-year-old has applied for special permission to enter the country.
Indian Wells and the Miami Open - two of the most prestigious tournaments on the tennis calendar
outside the Grand Slams - start on 6 and 20 March respectively. Djokovic says he will return to
the ATP tour in Dubai next week after claiming a record-extending 10th Australian Open title
and a record-equalling 22nd Grand Slam men's title last month.""".replace("\n", "")

In [6]:
summary = "Djokvic might be allowed to play in the US next month. Djokovic will play in Qatar next week after winning his 5th Grand slam."

In [7]:
score = mqag_model.score(candidate=summary, reference=document, num_questions=3, verbose=True)

Initialized Generation
Initialized Answering
Q1: What is the best title of this passage?
(1) [P(.|cand)=16.74%]	[P(.|ref)=34.21%]	Djokovic playing in Qatar
(2) [P(.|cand)=16.42%]	[P(.|ref)=28.77%]	Djokvic playing in Qatar
(3) [P(.|cand)=50.09%]	[P(.|ref)=2.81%]	Djokovic's 5th Grand slam
(4) [P(.|cand)=16.74%]	[P(.|ref)=34.21%]	Djokovic playing in Qatar
-------------------------------------------------------------------------------
Q2: Djokovic won his 5th Grand Slam _.
(1) [P(.|cand)=39.97%]	[P(.|ref)=36.00%]	in Qatar
(2) [P(.|cand)=7.03%]	[P(.|ref)=19.41%]	in China
(3) [P(.|cand)=20.08%]	[P(.|ref)=18.32%]	in Germany
(4) [P(.|cand)=32.91%]	[P(.|ref)=26.26%]	in England
-------------------------------------------------------------------------------
Q3: Djokovic might play in the US next month because _.
(1) [P(.|cand)=5.21%]	[P(.|ref)=93.53%]	he is allowed to play there
(2) [P(.|cand)=47.24%]	[P(.|ref)=1.64%]	he won his 5th Grand slam
(3) [P(.|cand)=0.31%]	[P(.|ref)=3.19%]	he won’t play 

In [8]:
print("KL-div    =", score['kl_div'])
print("Counting  =", score['counting'])
print("Hellinger =", score['hellinger'])
print("Total Var =", score['total_variation'])

KL-div    = 1.3974249426743093
Counting  = 0.6666666666666666
Hellinger = 0.37552807852247866
Total Var = 0.4932687332232793


# Multiple-choice Question Generation: `generate`

In [9]:
context = r"""
World number one Novak Djokovic says he is hoping for a "positive decision" to allow him
to play at Indian Wells and the Miami Open next month. The United States has extended
its requirement for international visitors to be vaccinated against Covid-19. Proof of vaccination
will be required to enter the country until at least 10 April, but the Serbian has previously
said he is unvaccinated. The 35-year-old has applied for special permission to enter the country.
Indian Wells and the Miami Open - two of the most prestigious tournaments on the tennis calendar
outside the Grand Slams - start on 6 and 20 March respectively. Djokovic says he will return to
the ATP tour in Dubai next week after claiming a record-extending 10th Australian Open title
and a record-equalling 22nd Grand Slam men's title last month.""".replace("\n", "")

In [10]:
questions = mqag_model.generate(context=context, do_sample=True, num_questions=3)
for i, question_item in enumerate(questions):
    print("------------------------------------")
    print(f"Q{i}: {question_item['question']}")
    print(f"A: {question_item['options'][0]}")
    print(f"B: {question_item['options'][1]}")
    print(f"C: {question_item['options'][2]}")
    print(f"D: {question_item['options'][3]}")

------------------------------------
Q0: How many Grand Slam men's titles has Djokovic won?
A: 22
B: 10
C: 20
D: 24
------------------------------------
Q1: Why is Djokovic hoping for a "positive decision"?
A: To be allowed to play at Indian Wellsand the Miami Open.
B: To play at the Miami Open.
C: To apply to stay at the US until at least 10 April.
D: To have his proof of vaccination.
------------------------------------
Q2: What does the author want to tell readers?
A: Djokovic won't leave the ATP tour in Dubai.
B: Djokovic will play at Indian Wells and the Miami Open next month.
C: Djokovic won't enter the country until at least 10 April.
D: The United States doesn't need foreign visitors to be vaccinated again.


# Multiple-choice Question Answering: `answer`

In [11]:
context = r"""Chelsea's mini-revival continued with a third victory in a row as they consigned struggling Leicester City to a fifth consecutive defeat.
Buoyed by their Champions League win over Borussia Dortmund, Chelsea started brightly and Ben Chilwell volleyed in from a tight angle against his old club.
Chelsea's Joao Felix and Leicester's Kiernan Dewsbury-Hall hit the woodwork in the space of two minutes, then Felix had a goal ruled out by the video assistant referee for offside.
Patson Daka rifled home an excellent equaliser after Ricardo Pereira won the ball off the dawdling Felix outside the box.
But Kai Havertz pounced six minutes into first-half injury time with an excellent dinked finish from Enzo Fernandez's clever aerial ball.
Mykhailo Mudryk thought he had his first goal for the Blues after the break but his effort was disallowed for offside.
Mateo Kovacic sealed the win as he volleyed in from Mudryk's header.
The sliding Foxes, who ended with 10 men following Wout Faes' late dismissal for a second booking, now just sit one point outside the relegation zone.
""".replace('\n', ' ')

question = "Who had a goal ruled out for offside?"
options  = ['Ricardo Pereira', 'Ben Chilwell', 'Joao Felix', 'The Foxes']

In [12]:
questions = [{'question': question, 'options': options}]
probs = mqag_model.answer(questions=questions, context=context)
print(probs[0])

[0.00145158 0.00460854 0.99049687 0.00344299]
